# Actividad Integradora 1

Realizado por Andrea Corona Arroyo A01366768

Link a Github: https://github.com/AndreaCorona523/MultiAgentes.git

Descripción del problema

¡Felicidades! Eres el orgulloso propietario de 5 robots nuevos y un almacén lleno de cajas. El dueño 
anterior del almacén lo dejó en completo desorden, por lo que depende de tus robots organizar las 
cajas en algo parecido al orden y convertirlo en un negocio exitoso.  

Cada  robot  está  equipado  con  ruedas  omnidireccionales  y,  por  lo  tanto,  puede  conducir  en  las 
cuatro direcciones. Pueden recoger cajas en celdas de cuadrícula adyacentes con sus 
manipuladores, luego llevarlas a otra ubicación e incluso construir pilas de hasta cinco cajas. Todos 
los robots están equipados con la tecnología de sensores más nueva que les permite recibir datos 
de sensores de las cuatro celdas adyacentes. Por tanto, es fácil distinguir si un campo está libre, es 
una pared, contiene una pila de cajas (y cuantas cajas hay en la pila) o está ocupado por otro robot. 
Los robots también tienen sensores de presión equipados que les indican si llevan una caja en ese 
momento.  

Lamentablemente,  tu  presupuesto  resultó  insuficiente  para  adquirir  un  software  de  gestión  de 
agentes múltiples de última generación. Pero eso no debería ser un gran problema ... ¿verdad? Tu 
tarea es enseñar a sus robots cómo ordenar su almacén. La organización de los agentes depende de 
ti, siempre que todas las cajas terminen en pilas ordenadas de cinco. 

Realiza la siguiente simulación: 
- Inicializa las posiciones iniciales de las K cajas. Todas las cajas están a nivel de piso, es decir, no hay pilas de cajas. 
- Todos los agentes empiezan en posición aleatorias vacías. 
- Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución: 
- Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas. 
- Número de movimientos realizados por todos los robots. 
- Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Cómo sería? Descríbela. 

## Imports

Antes de empezar a crear el modelo de los robots de limpieza con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`
- `matplotlib`

Para poder modelar usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que únicamente se permitirá un agente por celda, se utiliza el modo "SingleGrid".
from mesa.space import SingleGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd
import random
import operator

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Creación del Modelo

In [86]:
def obtenerAlmacen(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    colorsStacks = [0.30, 0.55, 0.65, 0.2, 0.83, 0.75]
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        if isinstance(cell_content, Box):
            grid[x][y] = cell_content.status
        elif isinstance(cell_content, RobotAgent):
            if cell_content.withBox:
                grid[x][y] = 0.4
            else:
                grid[x][y] = 0
        elif isinstance(cell_content, Stack):
            grid[x][y] = colorsStacks[cell_content.numBoxes]
            
        else:
            grid[x][y] = 1
    return grid

class RobotAgent(Agent):
    '''
    Representa a un robot.
    '''
    def __init__(self, unique_id, model):
        '''
        Agente representa un robot, se inicializa con un id, con el numero de movimientos realizados en 0, 
        y con su siguiente posición, que posee una caja, la caja, su posición objetivo y la pila objetivo como None.
        Así como una lista vacia de celdas visitadas.
        '''
        super().__init__(unique_id, model)
        self.numMovimientos = 0
        self.nextPos = None
        self.withBox = False
        self.box = None
        self.cellsVisited = []
        self.targetPos = None
        self.targetStackPos = None

    def detectBoxes(self):
        '''
        Método que detecta una caja en cualquiera de las cuatro celdas adyacentes.
        '''
        neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)
        
        #Verifica si alguno de los vecinos es una caja
        for neighbor in neighbours:
            if isinstance(neighbor, Box):
                return neighbor

    
    def pickUpBox(self, box):
        '''
        Método que recoje una caja del suelo y la carga en el robot.
        '''
        #Verifica que la caja no sea recogida por otro robot.
        if box.isAvailable():
            self.model.kill_agents.append(box)
            self.cellsVisited.clear()
            self.box = box
            self.nextPos = self.pos
            box.updateRobot(self.unique_id)
            self.withBox = True
            return True
        return False
        
    def dropBox(self, stackPosition):
        '''
        Método que deposita la caja en una pila.
        '''
        
        #Si la pila existe y no está vacía, deja la caja
        stack = self.model.returnStack(stackPosition)
        if not self.model.checkStackFull(stackPosition) and stack:
            self.box.updateRobot(-1)
            self.box.updateStack(stack.unique_id)

            stack.addBox()
            stack.updateStatus()

            self.withBox = False
            self.box = None
            return True
        
        #Si la está llena o no existe, manda crear la pila
        elif not stackPosition in self.model.getMovementsAgents(self) and self.model.grid.is_cell_empty(stackPosition) and not self.model.stacksFree():
            self.model.createStack = True
            self.model.stackNewPos = stackPosition
            return False
        else:
            return False
            

    def getPossibleMovements(self, position):
        '''
        Método que obtiene las celdas a las que es posible que se mueva el robot.
        '''
        neighborhood = self.model.grid.get_neighborhood(
                position,
                moore=False,
                include_center=False)
        
        neighbours = self.model.grid.get_neighbors(
            position,
            moore=False,
            include_center=False)
        
        #Obtiene celdas que no sean cajas y a las que no se vayana mover los otros robots
        cellsAvailable = [cell for cell in neighborhood if self.model.grid.is_cell_empty(cell) and not cell in self.model.getMovementsAgents(self)]
        return cellsAvailable
 
            
    def moveToRandom(self):
        '''
        Método que mueve aleatoriamente al robot en cualquiera de las 4 celdas adyacentes.
        '''
        possibleMoves = self.getPossibleMovements(self.pos)
        movementPossible = False
        if len(possibleMoves) > 0:
            approved = False
            idx = 0
            while not approved and idx < len(possibleMoves):
                cell = random.choice(possibleMoves)
                
                #Para evitar que el robot repita las mismas posiciones se verifica que no haya repetido la celda
                if not cell in self.cellsVisited:
                    self.nextPos = cell
                    self.cellsVisited.append(cell)
                    approved = True
                    movementPossible = True
                idx += 1
        
        #Si el robot ya no se puede mover, se queda en su lugar y se vacia la lista de celdas visitadas
        if not movementPossible:
            self.nextPos = self.pos
            self.cellsVisited.clear()
        
    
    def moveToPosition(self, position):
        '''
        Método que dirige al robot hacia una posición específica.
        '''
        distances = []
        possibleMovement = False
        
        if self.model.getManhattanDistance(self.pos, position) != 0:
            cellsAvailable = self.getPossibleMovements(self.pos)
            distances = self.model.getDistancesFromPosition(position, cellsAvailable)
            
            #Encuentra la posición que lo lleva a la pila más cercana más rápido y define esa como su siguiente casilla
            if len(distances) > 0:
                idx = 0
                while not possibleMovement and idx < len(distances):
                    newPos = distances[idx][1]
                    if not newPos in self.cellsVisited:
                        possibleMovement = True
                        self.nextPos = newPos
                        self.cellsVisited.append(newPos)
                    idx += 1
        
        #Si no es posible seguir el camino optimo, elige una opción aleatoria.
        if not possibleMovement:
            self.moveToRandom()
                            
    
            
    def step(self):
        '''
        En este método el agente realiza las acciones del agente por cada paso.
        '''
        
        
        #Si el robot no tiene una caja, busca por cajas en su celdas adyacentes
        if not self.withBox:
            box = self.detectBoxes()
            #Si detecta una caja intenta recogerla
            if box:
                
                #Si recogió una caja busca la pila más cercana y se dirige hacia ella
                if self.pickUpBox(box):
                    self.pickUpBox(box)
                    self.targetStackPos = self.model.getCloserStack(self.pos)
                    result = self.model.getCloserPositionOfStackPosition(self.pos, self.targetStackPos)
                    if result:
                        self.targetPos = result[1]
                    else:
                        self.moveToRandom()
            
        if self.withBox:
            
            #Si el robot tiene una caja, y ya llego a la posición y la pila sigue con espacio disponible, la deposita
            if self.pos == self.targetPos and not self.model.checkStackFull(self.targetStackPos):
                dropped = self.dropBox(self.targetStackPos)
                
                #Si no pudo depositar la caja y no hay pilas disponibles, espera en la misma posición
                if not dropped or not self.model.stacksFree():
                    self.nextPos = self.pos
                    
            #Si no está en la posición, la pila objetivo se llenó y hay pilas disponibles en otro lugar
            #Busca una nueva posición a la que dirigirse
            if self.pos != self.targetPos or self.model.checkStackFull(self.targetStackPos) or self.model.stacksFree():
                self.targetStackPos = self.model.getCloserStack(self.pos)
                result = self.model.getCloserPositionOfStackPosition(self.pos, self.targetStackPos)
                if result:
                    self.targetPos = result[1]
                    self.moveToPosition(self.targetPos)
                else:
                    self.moveToRandom()
        
        #Si el robot no recogió una caja, se mueve aleatoriamente
        if not self.withBox:
            self.moveToRandom()
        
        #Si se movio, se suma 1 a los movimientos
        if self.pos != self.nextPos:
            self.numMovimientos += 1
            
            
        
    def advance(self):
        '''
        Define el nuevo estado calculado del método step.
        '''
        
        self.model.grid.move_agent(self, self.nextPos)
        


class Stack(Agent):
    '''
    Representa a una pila de cajas.
    '''
    def __init__(self, unique_id, model):
        '''
        Agente representa una pila de cajas, se inicializa con un id,
        un estado "full" (lleno) de falso y un numero de cajas en 0.
        '''
        super().__init__(unique_id, model)
        self.full = False
        self.numBoxes = 0
        
    def updateStatus(self):
        '''
        En este método se actualiza el estado de la caja si está lleno
        '''
        if self.numBoxes == 5:
            self.full = True
            
    def addBox(self):
        '''
        En este método se suma uno al número de cajas.
        '''
        self.numBoxes += 1
        
    

class Box(Agent):
    '''
    Representa a una caja. 
    '''
    BOX = 0.9
    def __init__(self, unique_id, model):
        '''
        Agente representa una caja, se inicializa con un id y status que indica que contiene una caja.
        '''
        super().__init__(unique_id, model)
        self.status = self.BOX
        self.numRobot = -1 
        self.numStack = -1
    
    def isAvailable(self):
        '''
        Método que regresa si la caja está siendo cargada por otro robot.
        '''
        if self.numRobot == -1:
            return True
        return False
    
    def updateRobot(self, numRobot):
        '''
        En este método se actualiza el número del robot.
        '''
        self.numRobot = numRobot
    
    def updateStack(self, numStack):
        '''
        En este método se actualiza el número del stack.
        '''
        self.numStack = numStack

class StorageModel(Model):
    '''
    Define el modelo del almacen donde se encuentran los robots y las cajas. 
    '''
    def __init__(self, numAgents, height, width, numBoxes):
        self.grid = SingleGrid(height, width, False)
        self.width = width
        self.height = height
        self.schedule = SimultaneousActivation(self)
        self.running = True
        self.stacks = []
        self.kill_agents = []
        self.agents = []
        self.numBoxes = numBoxes
        self.numMovimientos = 0
        self.createStack = False
        self.stackNewPos = None
        
        #Crea un stack vacio inicial
        self.createNewStack((self.width//2, self.height//2))
        
        
        #Posicionar las cajas de forma aleatoria
        for cell in range(self.numBoxes):
            emptyCell = self.grid.find_empty()
            box = Box("00" + str(cell), self)
            box.status = box.BOX
            self.grid.place_agent(box, emptyCell)
            self.schedule.add(box)
        
        #Posicionar agentes de forma aleatoria
        for agent in range(numAgents):
            emptyCell = self.grid.find_empty()
            a = RobotAgent(agent, self)
            self.grid.place_agent(a, emptyCell)
            self.schedule.add(a)
            self.agents.append(a)
            
        # Aquí definimos con colector para obtener del almacén y el número de movimientos totales.
        self.datacollector = DataCollector(
            model_reporters={'Almacen': obtenerAlmacen},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'numMovimientos', None)},
        )
        
    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        Asimismo, eliminará los agentes de las cajas que hayan sido recogidas
        '''
        self.datacollector.collect(self)
        self.schedule.step()
        
        #Se retiran los agentes caja del grid
        for x in self.kill_agents:
            self.grid.remove_agent(x)
        
        #Se crea una nueva pila
        if self.createStack and self.grid.is_cell_empty(self.stackNewPos):
            self.createNewStack(self.stackNewPos)
            self.createStack = False
            self.stackNewPos = None
            
    def createNewStack(self, position):
        '''
        En este método se crea una nueva pila 
        '''
        if self.grid.is_cell_empty(position):
            stack = Stack("0" + str(len(self.stacks)), self)
            self.grid.place_agent(stack, position)
            self.schedule.add(stack)
            self.stacks.append(stack)
    
    def stacksFree(self):
        '''
        Este método verifica si hay alguna pila existente y que tenga espacio.
        '''
        free = False
        for stack in self.stacks:
            if not stack.full:
                free = True
        return free
    
    def checkStackFull(self, position):
        '''
        Método que verifica si una pila en una posición está lleno o no.
        '''
        for stack in self.stacks:
            if stack.pos == position:
                return stack.full
            
    def returnStack(self, position):
        '''
        Método que regresa la pila de una posicion dada.
        '''
        for stack in self.stacks:
            if stack.pos == position:
                return stack

    
    def allBoxesInStacks(self):
        '''
        En este método cuenta las cajas que han sido acomodadas en pila.
        '''
        num = 0
        for stack in self.stacks:
            num += stack.numBoxes
        if num == self.numBoxes:
            return True
        return False
    
    
    def getManhattanDistance(self, position1, position2):
        '''
        Método que obtiene la distancia Manhattan entre dos posiciones.
        '''
        return abs(position1[0] - position2[0]) + abs(position1[1] - position2[1])
    
    def getDistancesFromPosition(self, position, positions):
        '''
        Método que obtiene la distancia Manhattan entre una posición y un arreglo de posiciones 
        y las ordena de menor a mayor.
        '''
        distances = []
        for pos in positions:
            distances.append([self.getManhattanDistance(pos, position), pos])
        distances.sort(key=operator.itemgetter(0))
        return distances
    
    def getMovementsAgents(self, agentParam):
        '''
        Método que obtiene los movimientos de los agentes, menos del que está como parámetro.
        '''
        #Se almacenan los movimientos siguientes de los agentes para evitar que choquen
        return [agent.nextPos for agent in self.agents if agent != agentParam]
    
    def getCloserPositionOfStackPosition(self, agentPos, position):
        '''
        Método que obtiene la posicion disponible y la distancia mas cercana a una posicion determinada.
        '''
        
        #Se almacenan los movimientos siguientes de los agentes para evitar que choquen
        movements = [agent.nextPos for agent in self.agents if agent.pos != agentPos]
        
        neighborhood = self.grid.get_neighborhood(
                position,
                moore=False,
                include_center=False)
        
        
        neighbours = self.grid.get_neighbors(
            position,
            moore=False,
            include_center=False)
        
        neighborhoodFiltered = [cell for cell in neighborhood if self.grid.is_cell_empty(cell) and not cell in cell in movements]
        distances = self.getDistancesFromPosition(agentPos, neighborhoodFiltered)
        if len(distances) > 0:
            return distances[0]
        
        
    def getCloserStack(self, agentPos):
        '''
        Método que obtiene la posicion de la pila mas cercana o la posición más cercana para crear
        una pila nueva que esté cerca a alguna pila existente.
        '''
        #Se almacenan los movimientos siguientes de los agentes para evitar que choquen
        movements = [agent.nextPos for agent in self.agents if agent.pos != agentPos]
        
        #Ordena por la distancia desde el robot
        distancesAllStack = []
        for stack in self.stacks:
            distancesAllStack.append([self.getManhattanDistance(agentPos, stack.pos), stack])
        distancesAllStack.sort(key=operator.itemgetter(0))
        
        #Si hay pila disponible, regresa su posición
        if len(distancesAllStack) > 0:
            stacksAvailable = [stack for stack in distancesAllStack if not stack[1].full]
            if len(stacksAvailable) > 0:
                return stacksAvailable[0][1].pos
            else:
                
                #Si no hay pilas, regresa la posición más cercana para crear una pila nueva
                distancesStack = []
                for stack in distancesAllStack:
                    result = self.getCloserPositionOfStackPosition(agentPos, stack[1].pos)
                    if result:
                        distancesStack.append(result)
                distancesStack.sort(key=operator.itemgetter(0))
                if len(distancesStack) > 0:
                    return distancesStack[0][1]
                
        #Si no hay pilas se crea una nueva pila en el medio del almacén.
        return (self.width//2, self.height//2)
    
    def calculateTotalMovements(self):
        '''
        Método que calcula los movimientos totales de todos los agentes.
        '''
        for agent in self.agents:
            self.numMovimientos += agent.numMovimientos
        return self.numMovimientos
        
    

A continuación se corre el modelo:

In [96]:
# Definimos el tamaño del Grid
M = 15
N = 15

# Definimos el número de cajas
K = 25

# Definimos el número de agentes
NUM_AGENTS = 5

# Definimos tiempo máximo (segundos)
MAX_TIME = 0.5

# Registramos el tiempo de inicio y corremos el modelo
num_generations = 0

start_time = time.time()
model = StorageModel(NUM_AGENTS, M, N, K)
while time.time() - start_time < MAX_TIME and not model.allBoxesInStacks():
    model.step()
    num_generations += 1
model.step()
num_generations += 1
end_time = time.time()

Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [97]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando matplotlib

In [98]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.colormaps["gist_ncar"])

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=num_generations)

Para la siguiente simulación, se consideran los siguientes colores:
- Agente -> azul oscuro
- Agente con caja -> verde
- Caja -> rosa 
- Stack con 0 cajas -> verde claro
- Stack con 1 caja -> amarillo
- Stack con 2 cajas -> naranja
- Stack con 3 cajas -> azul claro
- Stack con 4 cajas -> morado
- Stack con 5 cajas (lleno) -> rojo

In [99]:
anim

Se imprimen los resultados obtenidos:

In [100]:
print("Tiempo: " + str(datetime.timedelta(seconds=(end_time - start_time))) + " / " + str(datetime.timedelta(seconds=(MAX_TIME))))
print("Número de movimientos realizados por todos los agentes: " ,  model.calculateTotalMovements())

Tiempo: 0:00:00.228829 / 0:00:00.500000
Número de movimientos realizados por todos los agentes:  1154


## Análisis




Hasta el momento la estrategia utilizada es que cada robot va recorriendo el almacén y conforme encuentra cajas las va llevando hacia la pila más cercana que encuentre, en caso de que no haya espacio en la fila, crea otra pila en una celda adyacente de la pila más cercana. Esta estrategia se apoya en las distancias de Manhattan para obtener las distancias más cortas y así evitar que el robot dé la menor cantidad de pasos. No obstante, se tiene que, en ocasiones, la ruta óptima puede ser bloqueada por otras cajas, pilas u otros robots, es por ello, que se tiene la opción de que si se encuentra atorado, realice un movimiento aleatorio para encontrar una ruta más corta. 

Lo anterior puede generar que el agente dé movimientos innecesarios cuando el objeto con el que se encontró fue un robot igualmente en movimiento. Por lo tanto, una estrategia de disminuir los movimientos sería implementar en los robots una regla de manera que al momento de dar el siguiente paso, revisen si están bloqueando el paso a un robot que lleva una caja. En caso de que lo estén, descartarían este paso y elegirían un movimiento diferente. De esta manera, los robots que llevan consigo una caja no tendrían su ruta óptima bloqueada, al menos para los robots. 

De igual manera, para evitar que su ruta óptima sea obstaculizada por cajas, se podría implementar que los robots sean capaces de dejar una caja en una posición que no sea una pila, para recoger la caja que les está impidiendo el camino. Posteriormente, se dirigirían a depositar esa caja en la pila más cercana, para posteriormente regresar por la caja que dejaron anteriormente.

Este último punto va de la mano de otra adaptación que se podría hacer, la cual es que los robots vayan almacenando en su memoria, las posiciones de las cajas que han encontrado pero no han podido recoger. De esta forma, los movimientos aleatorios se reducirían porque cada vez que el robot encuentra cajas, se dirigirá de las cajas hacia las pilas y viceversa, sin tomar movimientos aleatorios, hasta haber depositado todas las cajas que él ha detectado.

Esta estrategia alternativa permitiría que los movimientos de los robots fueran más precisos y dirigidos, por lo que, en consecuencia, se tendría una reducción en los tiempos de ejecución así como en la cantidad de movimientos totales realizados. 